In [4]:
import json
from pathlib import Path
import shutil

In [5]:
import cv2
from tqdm import tqdm

# Mirror left leaning poses to right leaning poses

In [6]:
angle_l_limit = 52  # Left leaning arbitrary limit

In [7]:
input_json_pose_file = '../data/filtered_pose.json'
input_json_pose_folder = '../data/wiki_crop_filtered_pose'

output_json_pose_mirrored_file = '../data/filtered_pose_mirrored.json'
output_pose_mirrored_folder = '../data/wiki_crop_filtered_mirrored_pose'

In [8]:
with open(input_json_pose_file, 'r') as f:
    records = json.load(f)

In [9]:
Path(output_pose_mirrored_folder).mkdir(exist_ok=True)

count = 0
for file_name in tqdm(records):
    values = records[file_name]
    
    # if not values['angle_l'] > angle_l_limit:
    if not values['angle_l'] > values['angle_r']:
        shutil.copy(f'{input_json_pose_folder}/{file_name}', f'{output_pose_mirrored_folder}/{file_name}')
        continue

    image = cv2.imread(f'{input_json_pose_folder}/{file_name}')
    _, width, _ = image.shape
    image = cv2.flip(image, 1)
    cv2.imwrite(f'{output_pose_mirrored_folder}/{file_name}', image)
    
    new_bbox = [
        width - values['bbox'][2], 
        values['bbox'][1], 
        width - values['bbox'][0], 
        values['bbox'][3]
    ]

    new_landmarks = {}
    for key, values in new_landmarks.items():
        new_landmarks[key] = [width - values[0], values[1]]
    
    # Flip the angles
    angle_l, angle_r = values['angle_r'], values['angle_l']
    
    values.update({
        'bbox': new_bbox,
        'landmarks': new_landmarks,
        'angle_l': angle_l,
        'angle_r': angle_r
    })
    
    count += 1

100%|██████████| 15351/15351 [00:30<00:00, 505.87it/s]


In [10]:
print(count)

8206


In [11]:
with open(output_json_pose_mirrored_file, 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

In [ ]:
# iterate over df
